# flagchat 套件用法示範

本套件主要將 openai ChatCompletion API 抽象化, 納入串流、function calling 功能, 並且將介面統一使用生成器產生回覆, 不論是否啟用串流模式, 都可用一致的方式取得回覆。另外, 也搭配 function calling 設計一個簡易的外掛系統。

## 事前準備

In [ ]:
# 設定環境變數
# openai 會使用 OPANAI_API_KEY
# 或是也可以直接設定 openai.api_key
# import openai
# openai.api_key = 你的 OpenAI API Key
!pip install python-dotenv openai googlesearch-python
from dotenv import load_dotenv
load_dotenv()

True

## 下載套件

In [ ]:
%%bash
git clone https://github.com/FlagTech/flagchat.git  flagchat
cd flagchat/
git pull

Already up to date.


fatal: destination path 'flagchat' already exists and is not an empty directory.


## 從模組匯入工具函式

In [ ]:
from flagchat import (
    get_reply,        # 輸入訊息串列傳回回覆
    chat,             # 輸入 system, user 發言取得回覆並會記錄對答歷史
    func_table,       # 記錄可用工具函式的參考表, 預設有 Google 搜尋函式
    set_backtrace,    # 設定記錄幾組對答 (預設：2)
    empty_history,    # 清除對答歷史
    set_verify_depth, # 設定驗證次數 (預設：3)
)

In [ ]:
set_backtrace(2)

2

In [ ]:
set_verify_depth(3)

3

## 單一問答測試

get_reply 可以會透過 function calling 機制使用 func_table 傳入的函式表格傳回回覆。模組內預設的 func_table 只有 Google 搜尋函式。

```python
get_reply(
    messages, # 訊息串列
    stream=False # 是否啟用串流模式
    func_table=None # 工具函式參考表
)
```
另外有選用的參數：

```python
mode='gpt-3.5-turbo' # 指定模型
debug=False # 是否要顯示除錯訊息, 包含訊息串列內容
```

In [ ]:
# 測試非串流方式 function_calling 功能
for chunk in get_reply(  # 不論是否串流回覆, 都以生成器統一函式介面
    [{"role":"user", "content":"2023 NBA 冠軍是誰？"}], # 訊息串列
    func_table=func_table):                           # 工具函式表
    print(chunk)         # 非串流模式只會生成一次

嘗試叫用：google_res(**{'user_msg': '2023 NBA 冠軍'})
根據已發生的事實，2023年NBA冠軍是丹佛金塊（Denver Nuggets）。


In [ ]:
# 測試串流方式 function_calling 功能
for chunk in get_reply(   # 不論是否串流回覆, 都以生成器統一函式介面
    [{"role":"user", "content":"2023 NBA 冠軍是誰？"}], # 訊息串列
    stream=True,                                      # 啟用串流模式
    func_table=func_table):                           # 工具函式表
    print(chunk, end='')  # 串流方式每次生成片段, 不換行才能接續內容

嘗試叫用：_google_res(**{
  "user_msg": "2023 NBA 冠軍是誰？"
})
2023年NBA冠軍是丹佛金塊。

In [ ]:
# 測試非串流、無 function calling 功能
for chunk in get_reply(
    [{"role":"user", "content":"2023 NBA 冠軍是誰？"}]):
    print(chunk)

作为人工智能助手，无法预测未来的体育比赛结果，包括2023年的NBA冠军。NBA比赛的结果取决于许多因素，包括球队的表现、球员的状态和伤病等。因此，只能在比赛结束后才能确定谁将赢得冠军。


In [ ]:
# 測試串流、無 function calling 功能
for chunk in get_reply(
    [{"role":"user", "content":"2023 NBA 冠軍是誰？"}],
    stream=True):
    print(chunk, end='')

作为AI模型，我不能预测未来的具体结果。因此，我无法准确回答2023年NBA冠军是谁。NBA比赛的结果取决于球队的表现、球员的状态和各种其他因素。最后的结果是由比赛实际情况决定的。

## 歷史紀錄測試

chat 是以 get_reply 函式為基礎, 加上對談歷史紀錄的功能, 可以使用 backtrace 設定要記錄的對談組數。

```python
chat(
    sys_msg, # system 角色發言
    user_msg, # user 角色發言
    stream=False, # 是否啟用串流模式
    func_table=func_table # 工具函式參考表 (預設是模組內建的參考表)
)
```
一樣可以使用選用的參數：

```python
mode='gpt-3.5-turbo' # 指定模型
debug=False # 是否要顯示除錯訊息, 包含訊息串列內容
```

In [ ]:
for chunk in chat(
    '小助理',                  # system 角色發言
    '2023 NBA 冠軍是誰？',      # user 角色發言
    True):                    # 使用串流模式
    print(chunk, end='')

嘗試叫用：_google_res(**{
  "user_msg": "2023 NBA 冠軍"
})
根據已知的資訊，2023年NBA冠軍是丹佛金塊隊。他們在2023年總決賽中以5-1的比數擊敗邁阿密熱火隊，贏得了球隊有史以來的第一個NBA總冠軍。

底下會因為有歷史紀錄而影響建議的搜尋關鍵字：

In [ ]:
for chunk in chat(
    '小助理',                 # system 角色發言
    '那 2022 呢？',           # user 角色發言 (會延續對答脈絡)
    True):                   # 使用串流模式
    print(chunk, end='')

嘗試叫用：_google_res(**{
  "user_msg": "2022 NBA冠軍是誰？"
})
依據已發生的資訊，2022年NBA冠軍是金州勇士隊。他們在2022年總決賽中以4-2的比數擊敗波士頓塞爾提克隊，贏得了球隊自2018年以來的第四個NBA總冠軍。

chat 會使用模組內預設的 func_table, 如果不想啟用, 可以加讓 func_table 參數值 None

In [ ]:
for chunk in chat(
    '小助理',               # system 角色發言
    '那 2022 呢？',         # user 角色發言
    True,                  # 串流模式
    None):                 # 不使用工具函式參考表 (因此不會搜尋)
    print(chunk, end='')

對不起，我之前的回答有誤。根據我們所知，作為一個語言模型，我們無法預測未來的事件。所以，我無法告訴您2022年的NBA冠軍是哪一隊。您可以關注NBA的賽事，等到真正的比賽結果公佈。

## 連續交談測試

以下是使用 chat 設計的聊天程式：

In [ ]:
empty_history()
sys_msg = input("你希望ㄟ唉扮演：")
if not sys_msg.strip(): sys_msg = '使用繁體中文的小助理'
print()
while True:
    msg = input("你說：")
    if not msg.strip(): break
    print(f"{sys_msg}：", end = "")
    for reply in chat(sys_msg, msg,
                      stream=True,
                      debug=True,
                      model='gpt-4',
                      verify=True
                      ):
        print(reply, end = "")
    print('\n')

你希望ㄟ唉扮演：

你說：是枝裕和和宮崎駿兩人個別的最新作品分別是哪一部
使用繁體中文的小助理：
--------------
model:gpt-4
function table:
	google_res
messages:
	{'role': 'user', 'content': '是枝裕和和宮崎駿兩人個別的最新作品分別是哪一部'}
	{'role': 'system', 'content': '使用繁體中文的小助理'}
--------------
嘗試叫用：google_res(**{'user_msg': '是枝裕和 最新作品'})

--------------
model:gpt-4
function table:
	google_res
messages:
	{'role': 'user', 'content': '是枝裕和和宮崎駿兩人個別的最新作品分別是哪一部'}
	{'role': 'system', 'content': '使用繁體中文的小助理'}
	{'role': 'assistant', 'content': None, 'function_call': <OpenAIObject at 0x78d3909fbe70> JSON: {
  "name": "google_res",
  "arguments": "{\n\"user_msg\": \"\u662f\u679d\u88d5\u548c \u6700\u65b0\u4f5c\u54c1\"\n}"
}}
	{'role': 'function', 'name': 'google_res', 'content': '以下為已發生的事實：\n標題：是枝裕和最新作品《怪物》七度參戰康城影展全球首映兩 ...\n摘要：Jun 6, 2023 — 是枝裕和最新作品《怪物》七度參戰康城影展全球首映兩千觀眾起立鼓掌長達9分半鐘！ 聞名世界的日本知名導演是枝裕和，曾多次勇闖海外影展，在日本國內外都受到相當大的肯定，這次他的最新電影作品《怪物》，攜手王牌編劇坂元裕二、已故傳奇音樂大師坂本龍一，三位神級大師共同打造，並在昨天晚上正式於第\xa0...\n\n標題：是枝裕和Hirokazu Koreeda的全部作品（70）\n摘要：舞伎家的料理人 (2023) [ 导演/ 制片人/ 编剧] 导演:

## 新增工具函式

以文字生圖為例

In [ ]:
import openai

用 Image API 設計一個文生圖的工具函式：

In [ ]:
def txt_to_img_url(prompt):
    response = openai.Image.create(
        prompt=prompt,                 # 描述文字
        n=1,                           # 張數
        size='1024x1024')              # 尺寸
    return response['data'][0]['url']

在工具函式表中新增項目, 生圖後不需要再送回給 AI 處理, 所以 chain 項目設為 False：

In [ ]:
func_table.append(
    {                    # 每個元素代表一個函式
        "chain": False,  # 生圖後不需要傳回給 API
        "func": txt_to_img_url,
        "spec": {        # function calling 需要的函式規格
            "name": "txt_to_img_url",
            "description": "可由文字生圖並傳回圖像網址",
            "parameters": {
                "type": "object",
                "properties": {
                    "prompt": {
                        "type": "string",
                        "description": "描述要產生圖像內容的文字",
                    }
                },
                "required": ["prompt"],
            },
        }
    }
)

NameError: ignored

測試看看是不是可以正確生圖？

In [ ]:
for chunk in chat('小助理', '我想要夕陽下海豚躍出海面的圖像', True):
    print(chunk)

嘗試叫用：txt_to_img_url(**{'prompt': '夕陽下海豚躍出海面的圖像'})
https://oaidalleapiprodscus.blob.core.windows.net/private/org-bKARbCV5aOcKJXm3SpwJwnXB/user-hwS8wMY6Z8ZzjiE3tcFcl4mM/img-tc2aFauikvuTWuwIJwIlJtDy.png?st=2023-07-12T03%3A22%3A18Z&se=2023-07-12T05%3A22%3A18Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-07-11T21%3A30%3A27Z&ske=2023-07-12T21%3A30%3A27Z&sks=b&skv=2021-08-06&sig=O9fQPEI9xq8EbruKZ/nAfFI5ebwpwse5n4kFtC4BYVM%3D
